In [1]:
import numpy as np
import pandas as pd
from itertools import combinations_with_replacement
from scipy.stats import chi2
import scipy.stats as stats
ds = [206,99,103,105,104,489,108]

In [2]:
df = pd.read_excel('df.xlsx')

# Filter to just KHV rows
khv = df[df['Ethnic'] == 'KHV206g'] 

# Filter to just CYP2B6 rows
cyp2c19 = khv[khv['Gene'] == 'CYP2C19']

# Get unique CYP2B6 alleles 
cyp2c19_alleles = set(cyp2c19['Star Allele 1'].unique()) | set(cyp2c19['Star Allele 2'].unique())

print(cyp2c19_alleles)

{1, 2, 3, 35, 13, 15, 17}


In [3]:
cyp2c19_alleles = list(cyp2c19_alleles)
cyp2c19table = pd.DataFrame(cyp2c19_alleles, columns=['CYP2C19'])
cyp2c19table['KHV'] = 0.0
print(cyp2c19table)

   CYP2C19  KHV
0        1  0.0
1        2  0.0
2        3  0.0
3       35  0.0
4       13  0.0
5       15  0.0
6       17  0.0


In [4]:
khv_cyp2c19 = df.query("Ethnic == 'KHV206g' & Gene == 'CYP2C19'")

print(khv_cyp2c19)
print(cyp2c19_alleles)
start = 0
for allele in cyp2c19_alleles:
  res = 0
  for index, row in khv_cyp2c19.iterrows():
    if allele == row['Star Allele 1']:
      res += row['Percentage per allele']
    if allele == row['Star Allele 2']:  
      res += row['Percentage per allele']
  cyp2c19table.at[start,'KHV'] = res
  start += 1


to_drop = cyp2c19table[cyp2c19table['KHV'] < 1]

# Drop those rows by index
cyp2c19table.drop(index=to_drop.index, inplace=True)
# Divide by 100  
print(cyp2c19table)

      Ethnic               Alleles  Percentage     Gene  Star Allele 1  \
413  KHV206g   CYP2C19*1/CYP2C19*1     40.7800  CYP2C19              1   
414  KHV206g  CYP2C19*1/CYP2C19*13      0.4854  CYP2C19              1   
415  KHV206g  CYP2C19*1/CYP2C19*15      0.4854  CYP2C19              1   
416  KHV206g  CYP2C19*1/CYP2C19*17      7.7670  CYP2C19              1   
417  KHV206g   CYP2C19*1/CYP2C19*2     35.9200  CYP2C19              1   
418  KHV206g   CYP2C19*1/CYP2C19*3      1.9420  CYP2C19              1   
419  KHV206g  CYP2C19*1/CYP2C19*35      0.9709  CYP2C19              1   
420  KHV206g  CYP2C19*2/CYP2C19*17      0.4854  CYP2C19              2   
421  KHV206g   CYP2C19*2/CYP2C19*2      6.7960  CYP2C19              2   
422  KHV206g   CYP2C19*2/CYP2C19*3      0.9709  CYP2C19              2   
423  KHV206g  CYP2C19*2/CYP2C19*35      0.4854  CYP2C19              2   

     Star Allele 2  Percentage per allele  
413              1               20.39000  
414             13     

In [5]:
temp = cyp2c19table['CYP2C19'].values.tolist()

ethnics = df['Ethnic'].unique()
ethnics =np.delete(ethnics, -1)
value_to_delete = 'KHV'
idx = np.where(ethnics == value_to_delete)[0][0]
ethnics = np.delete(ethnics, idx)
print(ethnics)


cyp2c19_rows = df[df['Gene'] == 'CYP2C19']

# Create bool series to check if allele in temp
allele1_matches = cyp2c19_rows['Star Allele 1'].isin(temp)
allele2_matches = cyp2c19_rows['Star Allele 2'].isin(temp)

# Filter for rows where either allele matches 
mask = allele1_matches | allele2_matches
cyp2c19_population = cyp2c19_rows[mask]
cyp2c19_population = cyp2c19_population[cyp2c19_population['Ethnic'] != 'KHV206g']

print(cyp2c19_population)#print(temp)

['CEU' 'CHB' 'CHS' 'JPT' 'SAS' 'YRI']
    Ethnic                Alleles  Percentage     Gene  Star Allele 1  \
124    CEU    CYP2C19*1/CYP2C19*1     36.3600  CYP2C19              1   
125    CEU   CYP2C19*1/CYP2C19*17     33.3300  CYP2C19              1   
126    CEU    CYP2C19*1/CYP2C19*2     18.1800  CYP2C19              1   
127    CEU    CYP2C19*1/CYP2C19*8      2.0200  CYP2C19              1   
128    CEU  CYP2C19*17/CYP2C19*17      3.0300  CYP2C19             17   
..     ...                    ...         ...      ...            ...   
182    YRI  CYP2C19*17/CYP2C19*35      0.9259  CYP2C19             17   
183    YRI   CYP2C19*2/CYP2C19*13      0.9259  CYP2C19              2   
184    YRI   CYP2C19*2/CYP2C19*17      5.5560  CYP2C19              2   
185    YRI    CYP2C19*2/CYP2C19*2      4.6300  CYP2C19              2   
186    YRI    CYP2C19*2/CYP2C19*9      0.9259  CYP2C19              2   

     Star Allele 2  Percentage per allele  
124              1               18.18000

In [6]:
for ethnic in ethnics:
        cyp2c19table[ethnic] = 0.0

cyp2c19table = cyp2c19table.reset_index(drop=True)

print(cyp2c19table)

   CYP2C19       KHV  CEU  CHB  CHS  JPT  SAS  YRI
0        1  64.56535  0.0  0.0  0.0  0.0  0.0  0.0
1        2  25.72685  0.0  0.0  0.0  0.0  0.0  0.0
2        3   1.45645  0.0  0.0  0.0  0.0  0.0  0.0
3       17   4.12620  0.0  0.0  0.0  0.0  0.0  0.0


In [7]:
start = 0
for ethnic in ethnics:
    start = 0
    for starallele in temp:
        res = 0

        for index, row in cyp2c19_population.iterrows():
            if row['Ethnic'] == ethnic:
                if starallele == row['Star Allele 1']:
                    res += row['Percentage per allele']
                if starallele == row['Star Allele 2']:  
                    res += row['Percentage per allele']
                cyp2c19table.at[start,ethnic] = res
        #print(start)    
        start += 1
    cyp2c19table[ethnic] = cyp2c19table[ethnic].round(1)

cyp2c19table['KHV'] = cyp2c19table['KHV'].round(1)
cyp2c19table.to_csv('frq/cyp2c19.csv')
print(cyp2c19table)

   CYP2C19   KHV   CEU   CHB   CHS   JPT   SAS   YRI
0        1  64.6  63.1  59.2  59.0  60.1  47.9  49.1
1        2  25.7  13.1  33.5  35.2  32.2  35.7  16.7
2        3   1.5   0.0   4.4   4.8   7.2   1.2   0.0
3       17   4.1  22.2   2.4   1.0   0.5  13.6  24.5


In [8]:
init = {'CEU': [0], 'CHB': [0], 'CHS': [0], 'JPT': [0], 'SAS': [0], 'YRI': [0]}
gst = pd.DataFrame(init, dtype=float)

h_s_k = 1.0
for index,value in cyp2c19table.iterrows():
    h_s_k -= (cyp2c19table.at[index,'KHV']/100) * (cyp2c19table.at[index,'KHV']/100)

for idx in range(0,6):
    value = 0
    h_t = 1.0
    h_s = 1.0
    for id in range(0,4):
        h_t -= ((cyp2c19table.iloc[id,idx + 2] + cyp2c19table.iloc[id,1])/200) * ((cyp2c19table.iloc[id,idx + 2] + cyp2c19table.iloc[id,1])/200)
        h_s -= (cyp2c19table.iloc[id,idx + 2]/100) * (cyp2c19table.iloc[id,idx + 2]/100)
    h_s = (h_s + h_s_k)/2
    gst.iloc[0,idx] = (h_t - h_s)/h_t # You can modify the value here'''
gst = gst.round(4)
gst.index = ['CYP2C19']
gst.to_csv('gst/gst_cyp2c19.csv')
print(gst)

            CEU     CHB     CHS     JPT     SAS     YRI
CYP2C19  0.0228  0.0048  0.0068  0.0051  0.0202  0.0302


In [9]:
test = list(cyp2c19table['CYP2C19'])
test = ['CYP2C19*' + str(int(item)) for item in test]

def generate_gene_combinations(alleles):
    res = []
    length = len(alleles)
    for x in range(0,length):
        for y in range(x,length):
            lol = f'{alleles[x]}/{alleles[y]}'
            res.append(lol)

    return res

print(generate_gene_combinations(test))
gene_combinations = generate_gene_combinations(test)

degrees_of_freedom = len(gene_combinations) - 1
print(gene_combinations)

['CYP2C19*1/CYP2C19*1', 'CYP2C19*1/CYP2C19*2', 'CYP2C19*1/CYP2C19*3', 'CYP2C19*1/CYP2C19*17', 'CYP2C19*2/CYP2C19*2', 'CYP2C19*2/CYP2C19*3', 'CYP2C19*2/CYP2C19*17', 'CYP2C19*3/CYP2C19*3', 'CYP2C19*3/CYP2C19*17', 'CYP2C19*17/CYP2C19*17']
['CYP2C19*1/CYP2C19*1', 'CYP2C19*1/CYP2C19*2', 'CYP2C19*1/CYP2C19*3', 'CYP2C19*1/CYP2C19*17', 'CYP2C19*2/CYP2C19*2', 'CYP2C19*2/CYP2C19*3', 'CYP2C19*2/CYP2C19*17', 'CYP2C19*3/CYP2C19*3', 'CYP2C19*3/CYP2C19*17', 'CYP2C19*17/CYP2C19*17']


In [10]:
observed = pd.DataFrame(gene_combinations, columns=['Gene'])
new_columns = ['KHV', 'CEU', 'CHB', 'CHS', 'JPT', 'SAS', 'YRI']
for column in new_columns:
    observed[column] = 0.0

print(observed)

                    Gene  KHV  CEU  CHB  CHS  JPT  SAS  YRI
0    CYP2C19*1/CYP2C19*1  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1    CYP2C19*1/CYP2C19*2  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2    CYP2C19*1/CYP2C19*3  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3   CYP2C19*1/CYP2C19*17  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4    CYP2C19*2/CYP2C19*2  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5    CYP2C19*2/CYP2C19*3  0.0  0.0  0.0  0.0  0.0  0.0  0.0
6   CYP2C19*2/CYP2C19*17  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7    CYP2C19*3/CYP2C19*3  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8   CYP2C19*3/CYP2C19*17  0.0  0.0  0.0  0.0  0.0  0.0  0.0
9  CYP2C19*17/CYP2C19*17  0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [11]:
l = len(khv_cyp2c19)
for x in range(0,l):
    if khv_cyp2c19.iloc[x,1] in gene_combinations:
        gene_to_update = khv_cyp2c19.iloc[x,1]
        new_value = khv_cyp2c19.iloc[x,2]
        observed.loc[observed['Gene'] == gene_to_update, 'KHV'] = new_value

length = len(cyp2c19_population)
for x in range(0,length):
    if cyp2c19_population.iloc[x,1] in gene_combinations:
        gene_to_update = cyp2c19_population.iloc[x,1]
        pop_to_update = cyp2c19_population.iloc[x,0]
        new_value = cyp2c19_population.iloc[x,2]
        observed.loc[observed['Gene'] == gene_to_update, pop_to_update ] = new_value 

for x in range(0,7):
    for y in range(0,len(gene_combinations)):
        observed.iloc[y,x+1] = observed.iloc[y,x+1] * ((ds[x] * 1.0)/ 100)

observed = observed.round(0)
print(observed)

                    Gene   KHV   CEU   CHB   CHS   JPT    SAS   YRI
0    CYP2C19*1/CYP2C19*1  85.0  36.0  34.0  31.0  39.0  113.0  22.0
1    CYP2C19*1/CYP2C19*2  81.0  18.0  48.0  54.0  39.0  159.0  18.0
2    CYP2C19*1/CYP2C19*3  12.0   0.0   4.0   6.0   8.0    4.0   0.0
3   CYP2C19*1/CYP2C19*17   6.0  33.0   1.0   2.0   0.0   73.0  29.0
4    CYP2C19*2/CYP2C19*2  15.0   2.0   7.0   8.0  11.0   74.0   5.0
5    CYP2C19*2/CYP2C19*3   4.0   0.0   4.0   4.0   5.0    6.0   0.0
6   CYP2C19*2/CYP2C19*17   2.0   4.0   3.0   0.0   1.0   31.0   6.0
7    CYP2C19*3/CYP2C19*3   0.0   0.0   0.0   0.0   1.0    0.0   0.0
8   CYP2C19*3/CYP2C19*17   0.0   0.0   1.0   0.0   0.0    2.0   0.0
9  CYP2C19*17/CYP2C19*17   0.0   3.0   0.0   0.0   0.0   12.0   8.0


In [12]:
hypo = pd.DataFrame(gene_combinations, columns=['Gene'])
for column in new_columns:
    hypo[column] = 0.0

allele = cyp2c19table.iloc[:,0].tolist()

for x in range(0,7):
    temp = cyp2c19table.iloc[:,x + 1].tolist()
    for allele1 in range(0,len(temp)):
        for allele2 in range(allele1,len(temp)):
            if allele1 == allele2:
                new_value = (temp[allele1] / 100) * (temp[allele2] / 100)
            elif allele1 != allele2:
                new_value = 2 * (temp[allele1] / 100) * (temp[allele2] / 100)
            gene = f"CYP2C19*{int(allele[allele1])}/CYP2C19*{int(allele[allele2])}"
            
            row_index = hypo.index[hypo['Gene'] == gene].tolist()[0]
            hypo.iloc[row_index, x+1] = new_value

print(hypo)

                    Gene       KHV       CEU       CHB       CHS       JPT  \
0    CYP2C19*1/CYP2C19*1  0.417316  0.398161  0.350464  0.348100  0.361201   
1    CYP2C19*1/CYP2C19*2  0.332044  0.165322  0.396640  0.415360  0.387044   
2    CYP2C19*1/CYP2C19*3  0.019380  0.000000  0.052096  0.056640  0.086544   
3   CYP2C19*1/CYP2C19*17  0.052972  0.280164  0.028416  0.011800  0.006010   
4    CYP2C19*2/CYP2C19*2  0.066049  0.017161  0.112225  0.123904  0.103684   
5    CYP2C19*2/CYP2C19*3  0.007710  0.000000  0.029480  0.033792  0.046368   
6   CYP2C19*2/CYP2C19*17  0.021074  0.058164  0.016080  0.007040  0.003220   
7    CYP2C19*3/CYP2C19*3  0.000225  0.000000  0.001936  0.002304  0.005184   
8   CYP2C19*3/CYP2C19*17  0.001230  0.000000  0.002112  0.000960  0.000720   
9  CYP2C19*17/CYP2C19*17  0.001681  0.049284  0.000576  0.000100  0.000025   

        SAS       YRI  
0  0.229441  0.241081  
1  0.342006  0.163994  
2  0.011496  0.000000  
3  0.130288  0.240590  
4  0.127449  0.027889

In [13]:
for x in range(0,7):
    for y in range(0,len(gene_combinations)):
        hypo.iloc[y,x+1] = hypo.iloc[y,x+1] * ((ds[x] * 1.0))
        if hypo.iloc[y,x+1] < 1:
            hypo.iloc[y,x+1] = 1
hypo = hypo.round(0)
print(hypo)

                    Gene   KHV   CEU   CHB   CHS   JPT    SAS   YRI
0    CYP2C19*1/CYP2C19*1  86.0  39.0  36.0  37.0  38.0  112.0  26.0
1    CYP2C19*1/CYP2C19*2  68.0  16.0  41.0  44.0  40.0  167.0  18.0
2    CYP2C19*1/CYP2C19*3   4.0   1.0   5.0   6.0   9.0    6.0   1.0
3   CYP2C19*1/CYP2C19*17  11.0  28.0   3.0   1.0   1.0   64.0  26.0
4    CYP2C19*2/CYP2C19*2  14.0   2.0  12.0  13.0  11.0   62.0   3.0
5    CYP2C19*2/CYP2C19*3   2.0   1.0   3.0   4.0   5.0    4.0   1.0
6   CYP2C19*2/CYP2C19*17   4.0   6.0   2.0   1.0   1.0   47.0   9.0
7    CYP2C19*3/CYP2C19*3   1.0   1.0   1.0   1.0   1.0    1.0   1.0
8   CYP2C19*3/CYP2C19*17   1.0   1.0   1.0   1.0   1.0    2.0   1.0
9  CYP2C19*17/CYP2C19*17   1.0   5.0   1.0   1.0   1.0    9.0   6.0


In [14]:
hihi = {"KHV": [0.0], "CEU": [0.0], "CHB": [0.0], "CHS": [0.0], "JPT": [0.0], "SAS": [0.0], "YRI": [0.0]}
chi_square = pd.DataFrame(hihi)
print(chi_square)
for x in range(0,7):
    res = 0
    for y in range(0,len(hypo)):
        chi_square_value = ((hypo.iloc[y,x+1] - observed.iloc[y,x+1]) * (hypo.iloc[y,x+1] - observed.iloc[y,x+1])) / (hypo.iloc[y,x+1])
        res += chi_square_value

    p_value = 1 - stats.chi2.cdf(res, degrees_of_freedom)
    chi_square.iloc[0,x] = p_value 

chi_square = chi_square.round(4)

chi_square.index = ['CYP2C19']
print(chi_square)

   KHV  CEU  CHB  CHS  JPT  SAS  YRI
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
            KHV     CEU     CHB    CHS     JPT     SAS    YRI
CYP2C19  0.0015  0.6537  0.5589  0.337  0.9575  0.1584  0.538


In [ ]:
print(cyp2c19table)
def f(row):
    return "CYP2C19*" + str(row)
cyp2c19table["CYP2C19"] = cyp2c19table["CYP2C19"].apply(f)
cyp2c19table.rename(columns={"CYP2C19": "Gene"}, inplace=True)
cyp2c19table.to_excel("heatmap/cyp2c19table.xlsx")